# PostgreSQL Data Governance Demonstration
This notebook demonstrates how to implement **data governance** principles using PostgreSQL. It covers user management, access control, and best practices for securing sensitive data.

We will work with a sample **patients database**, ensuring compliance with privacy and security regulations.


## Database Connection Setup and Auto-Creation
If the database does not exist, it will be created automatically.

In [ ]:
from sqlalchemy import create_engine
import psycopg2

# PostgreSQL connection settings
PG_ADDR = 'localhost'  # Server address
PG_PORT = '5432'       # PostgreSQL port
PG_USER = 'postgres'   # Admin user
PG_PASW = 'secure_password'  # Secure password
PG_DBNA = 'healthcare_db'  # Database name

# Connect to default PostgreSQL database to check and create if needed
default_engine = create_engine(f'postgresql://{PG_USER}:{PG_PASW}@{PG_ADDR}:{PG_PORT}/postgres')

with default_engine.connect().execution_options(autocommit=True) as conn:
    result = conn.execute("SELECT 1 FROM pg_database WHERE datname = %s;", (PG_DBNA,))
    exists = result.scalar()
    
    if not exists:
        conn.execute(f"CREATE DATABASE {PG_DBNA};")
        print(f"Database '{PG_DBNA}' created successfully!")

# Now connect to the newly created database
engine = create_engine(f'postgresql://{PG_USER}:{PG_PASW}@{PG_ADDR}:{PG_PORT}/{PG_DBNA}')
print(f"Connected to database '{PG_DBNA}'.")

## Creating the Patients Table
Let's create a new table to store patient information. This table includes sensitive data, so we will ensure proper access control.

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    conn.execute('''DROP TABLE IF EXISTS patients;''')
    conn.execute('''
        CREATE TABLE patients (
            id SERIAL PRIMARY KEY,
            full_name VARCHAR(100),
            date_of_birth DATE,
            country VARCHAR(50),
            diagnosis VARCHAR(100),
            insurance_amount DECIMAL(10,2)
        );
    ''')
print("Table 'patients' created successfully!")

## Inserting Sample Patient Data
Now, we will populate the **patients** table with fictitious data.

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Melissa Clark', '1935-04-04', 'Svalbard & Jan Mayen Islands', 'Heart Disease', 3370.96);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Wendy Taylor', '2018-04-11', 'Cambodia', 'Heart Disease', 1393.28);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Jennifer Lopez', '1999-08-09', 'Czech Republic', 'Cancer', 1788.08);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Steven Price', '1973-10-30', 'Ethiopia', 'Heart Disease', 4709.11);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Michael Thomas', '2009-06-01', 'United States of America', 'Cancer', 4859.27);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Tracy Morris', '1997-09-20', 'French Guiana', 'Diabetes', 1957.53);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Patrick Pacheco', '1984-07-12', 'Israel', 'Hypertension', 3688.64);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Valerie Dixon', '1987-08-29', 'Belize', 'Hypertension', 2058.29);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Kathryn Atkinson', '2015-08-30', 'Guyana', 'Diabetes', 3004.28);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Terrence Kidd', '1965-11-27', 'Mauritius', 'Heart Disease', 2205.32);")
print("Sample patient data inserted successfully!")

## Managing User Access
We will now create three new users with different roles and apply access restrictions.

In [ ]:
users = ['nurse', 'consultant', 'doctor']

with engine.connect().execution_options(autocommit=True) as conn:
    for user in users:
        conn.execute(f"DROP USER IF EXISTS {user};")
        conn.execute(f"CREATE USER {user} WITH PASSWORD 'password123';")
print("Users created successfully!")

## Revoking Privileges from Admin User
To enforce stricter data governance, we will revoke all privileges on the `patients` table from the **postgres** admin user.

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    conn.execute("REVOKE ALL PRIVILEGES ON TABLE patients FROM postgres;")
print("Privileges revoked from 'postgres' user!")

## Verifying Permissions
Let's check which users currently have access to the `patients` table.

In [ ]:
with engine.connect() as conn:
    result = conn.execute(
        "SELECT grantee, privilege_type FROM information_schema.role_table_grants "
        "WHERE table_name = 'patients';"
    )
    for row in result:
        print(row)
print("Current access privileges displayed!")